# Torontonian sampling example

*Author: Brajesh Gupt*

This Jupyter notebook uses the Torontonian Fortran library via Python to generate samples from Torontonians.

## Using the Fortran library

The Torontonian sampling Fortran library can be used either via Fortran, or via the Python interface.

### Interfacing via Fortran

If using the library via Fortran, no external dependencies are required. Simply run
```bash
make fortran
```
in the top level directory. The Fortran modules will be compiled, and the modules stored in the directory `include`. To use the module with your own Fortran, simply include the `use torontonian_samples` at the top of the program, and compile the commands
```bash
gfortran -o program program.f90  /path/to/include/*.o -I/path/to/include/
```

See the file `examples/fortran_example.f90` for an example program that uses the Torontonian sampling library.

### Interfacing via Python

To compile the library for use with Python, `NumPy` is required to be installed. This can be installed via `pip`L
```bash
pip install numpy
```
Then, simply run
```bash
make python
```
in the top level directory to compile the Python library. The library `torontonian_samples.cpython-*-.so` will be created, which can then be imported in Python via `import torontonian_samples`.

## Example usage

Import the library:

In [4]:
import torontonian_samples as tor

Import NumPy and random:

In [5]:
import numpy as np
import random

Import the Strawberry Fields package, to enable us to simulate a Guassian Boson Sampling system. This involves initialising a 10-mode system, squeezing each mode by the same magnitude $s$, and then applying a random interferometer.

In [44]:
import strawberryfields as sf
from strawberryfields.ops import *
from strawberryfields.utils import random_interferometer
from strawberryfields.backends.shared_ops import changebasis

In [45]:
l = 10
s = np.arcsinh(1.0)
U = random_interferometer(l)
    
eng, q = sf.Engine(10)
with eng:
    for i in range(l):
        Sgate(s) | q[i]
    Interferometer(U) | q
    
state = eng.run('gaussian')

The resulting vector of means (in this case, the displacement is 0):

In [49]:
C = changebasis(10)
r = C @ state.means()
r

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

The resulting covariance matrix:

In [50]:
cov = C @ state.cov() @ C.T

Note that we perform a change of basis operation, as the Torontonian sampling library expects the covariance in the form $(x_1, p_1, x_2, p_2, \dots)$.

Now we call `torontonian_samples.generatesample`, a Fortran subroutine, as follows to generate a sample: 

In [51]:
tor.torontonian_samples.generatesample(covmat=tormat,mean=r,seed=random.randint(0,10**6),n_sample=l)

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int64)

Now the above function can be called multiple times in a loop to generate as many samples as your heart desires.

In [53]:
samples=[]
for i in range(20):
    tmp = list(tor.torontonian_samples.generatesample(covmat=tormat,mean=r,seed=random.randint(0,10**6),n_sample=l))
    samples.append(tmp)

In [54]:
samples

[[0, 1, 0, 1, 0, 1, 0, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
 [1, 1, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 1, 1, 1, 1, 0, 0]]